# MTS Generation with GAN.
In this Notebook, we will try to generate Multivariate Time Series (MTS) with a simple GAN. This GAN will be only composed by:
- A Generator which will ONLY take a vector as input and try to generate a MTS.
- A Discriminator which will try to discriminate between the Fake and the Real Sample.

This Notebook will train this Simple GAN on Simulated Data.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from configs.Metric import Metric
from configs.SimulatedData import Proposed

# Load the Data.

In [ ]:
config = Proposed()

SIMULATED_DATA_PATH = "data/simulated_dataset/01 - Source Domain.h5"
NOISE_DIM= 64

SEQUENCE_LENGTH = config.sequence_lenght_in_sample
GRANUARITY = config.granularity
OVERLAP = config.overlap
BS = config.batch_size
EPOCHS = config.epochs
NUM_SEQUENCE_TO_GENERATE = config.met_params.sequence_to_generate


In [ ]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

gpus

In [ ]:
def convert_dataframe_to_tensorflow_sequences(df:pd.DataFrame, sequence_lenght_in_sample, granularity, shift_between_sequences, batch_size, shuffle=True):
    sequence_lenght = int(sequence_lenght_in_sample*granularity)

    dset = tf.data.Dataset.from_tensor_slices(df.values)
    dset = dset.window(sequence_lenght , shift=shift_between_sequences, stride=granularity).flat_map(lambda x: x.batch(sequence_lenght_in_sample, drop_remainder=True))

    if shuffle:
        dset= dset.shuffle(256)

    dset = dset.batch(batch_size, drop_remainder=True)

    dset = dset.cache().prefetch(10)

    return dset

In [ ]:
df_simulated = pd.read_hdf(SIMULATED_DATA_PATH)
dset_simulated = convert_dataframe_to_tensorflow_sequences(
    df_simulated, 
    SEQUENCE_LENGTH, 
    GRANUARITY, 
    int(OVERLAP* SEQUENCE_LENGTH),
    BS
)

## Verify if it is correct

In [ ]:
sequence = next(iter(dset_simulated))[0]
print(sequence.shape)

plt.figure(figsize=(18, 5))
plt.title("Simulated Sequence.")
for i in range(sequence.shape[1]):
    plt.plot(sequence[:, i], label=f'feat {i+1}')
plt.grid()
plt.legend()
plt.show()

## Make Model Architectures.

In [ ]:
from tensorflow.keras import layers

def make_generator(noise_shape:tuple, n_signals:int, seq_length:int):
    _model = tf.keras.Sequential()

    _model.add(tf.keras.Input(noise_shape))

    _model.add(tf.keras.layers.Dense(n_signals*seq_length//8))
    _model.add(layers.BatchNormalization())
    _model.add(layers.LeakyReLU())

    _model.add(tf.keras.layers.Reshape((seq_length//8, n_signals)))

    _model.add(tf.keras.layers.Conv1DTranspose(256, 5, 1, padding='same'))
    _model.add(layers.BatchNormalization())
    _model.add(layers.LeakyReLU())

    _model.add(tf.keras.layers.Conv1DTranspose(128, 5, 2, padding='same'))
    _model.add(layers.BatchNormalization())
    _model.add(layers.LeakyReLU())

    _model.add(tf.keras.layers.Conv1DTranspose(128, 5, 2, padding='same'))
    _model.add(layers.BatchNormalization())
    _model.add(layers.LeakyReLU())

    _model.add(tf.keras.layers.Conv1DTranspose(128, 5, 2, padding='same'))
    _model.add(layers.BatchNormalization())
    _model.add(layers.LeakyReLU())

    _model.add(tf.keras.layers.Conv1DTranspose(n_signals, 3, 1, padding='same'))
    _model.add(layers.BatchNormalization())
    _model.add(layers.LeakyReLU())

    return _model

generator = make_generator(NOISE_DIM, sequence.shape[1], SEQUENCE_LENGTH)
generator.summary()

In [ ]:
def make_discriminator(seq_length:int, n_feat:int):
    _input = tf.keras.Input((seq_length, n_feat))
    x = tf.keras.layers.Conv1D(128, 5, 2, padding='same')(_input)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = tf.keras.layers.Conv1D(128, 5, 2, padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    _output = layers.Dense(1)(x)

    model = tf.keras.Model(_input, _output)
    early_predictor = tf.keras.Model(_input, x)

    return model, early_predictor

discriminator, early_predictor = make_discriminator(SEQUENCE_LENGTH, sequence.shape[1])

### Plot a Sequence.

In [ ]:
seed = tf.random.normal([NUM_SEQUENCE_TO_GENERATE, NOISE_DIM])
generated = generator(seed)

# after_training_generations
def plot_several_generations(generations:np.ndarray, nvertical:int=3, nhoriz:int=3):

    legend = [f"feat {j}" for j in range(generations.shape[-1])]

    plt.figure(figsize=(18, 10))
    plt.suptitle("Several Generations")

    for i in range(nvertical* nhoriz):
        ax = plt.subplot(nvertical, nhoriz, i+ 1)
        ax.set_title(f"sequence {i+1}")

        plt.plot(generations[i])
        ax.grid(True)
        plt.legend(legend)

    plt.tight_layout()
    plt.show()

plot_several_generations(generated) 


### Test Discriminator

In [ ]:
decision = discriminator(generated)
decision

In [ ]:
def similarity_loss(extracted_features:np.ndarray):
    anchor = extracted_features[0]
    return tf.exp(-(tf.norm(extracted_features[1:]- anchor)))


## Define losses

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


## Corelation metric

In [ ]:
def optimized_cov(a,v):
    nan_mean_a = np.nanmean(a, axis=1).reshape((-1,1))
    nan_mean_b = np.nanmean(v, axis=1).reshape((-1,1))
    return np.nansum((a- nan_mean_a)* (v- nan_mean_b), axis=1)

def mean_difference(a,v):
    return np.nanmean(a) - np.nanmean(v)

def optimized_windowed_cov(a, v, beta=Metric.mean_senssibility_factor):
    if a.shape[1] > v.shape[1]:
        _a, _v = v, a 
    else: 
        _a, _v = a, v

    n = _a.shape[1]
    corrs = []

    for k in range(_v.shape[1] - _a.shape[1]):
        __v = _v[:, k: n+k]
        # Compute the covariance 
        augmented_cov = optimized_cov(_a,__v)+ beta* mean_difference(_a,__v)

        corrs.append(augmented_cov)
        
    return np.array(corrs)

def signature_on_batch(x:np.ndarray, ins:list, outs:list, sig_seq_len:int):
    """Compute the signature from a given batch of MTS sequences `x`

    Args:
        x (np.ndarray): the batch
        ins (list): input columns
        outs (list): output label solumns
        sig_seq_len (int): the desired signature length

    Returns:
        np.ndarray: the min, max, mean signature.
    """
    sigs = []
    shift = sig_seq_len//2
    childrens = x[:, shift:-shift]

    for _in in ins:
        for _out in outs:
            c1 = x[:, :, _in]
            c2 = childrens[:, :, _out]
            
            sig = optimized_windowed_cov(c1, c2)

            sigs.append(sig)

    mins = np.min(sigs, axis=-1)
    maxs = np.max(sigs, axis=-1)
    means= np.mean(sigs, axis=-1)

    signatures = np.stack([mins, maxs, means], axis=-1)

    return signatures

def signature_metric(source_sig:np.ndarray, target_sig:np.ndarray):
    # Shape: (n_features, sign_seq_lenght, 3)
    min_source = source_sig[0]
    max_source = source_sig[1]
    mean_source = source_sig[2]

    min_target = target_sig[0]
    max_target = target_sig[1]
    mean_target = target_sig[2]

    mean_differences = np.mean(mean_target- mean_source)
    area_source = np.mean(max_source- min_source)
    area_target = np.mean(max_target- min_target)

    met = np.power(mean_differences, 2) + Metric.noise_senssitivity*np.power(area_target- area_source, 2)

    return met

In [ ]:
## Generate the reference signature.

real_sequence_batch = next(iter(dset_simulated))

real_batch_signature= signature_on_batch(real_sequence_batch, config.met_params.ins, config.met_params.outs, config.met_params.signature_length)
generated_batch_signature= signature_on_batch(generated, config.met_params.ins, config.met_params.outs, config.met_params.signature_length)

signature_metric(real_batch_signature, generated_batch_signature)

## Optimizers

In [ ]:
generator_optimizer = tf.keras.optimizers.RMSprop(1e-4)
discriminator_optimizer = tf.keras.optimizers.RMSprop(2e-6)    

# generator_optimizer = tf.keras.optimizers.Adam(1e-4)
# discriminator_optimizer = tf.keras.optimizers.Adam(1e-6)    

## Define metrics 

In [ ]:
generator_metric = tf.keras.metrics.Mean()
discriminator_metric = tf.keras.metrics.Mean()
similarity_metric = tf.keras.metrics.Mean()
correlation_metric = tf.keras.metrics.Mean()

## Tensorboard logs.

In [ ]:
from datetime import datetime
import io

date_str = datetime.now().strftime('%Y-%m-%d_%H_%M_%S')

BASE_DIR = f"logs/{date_str}"
TRAIN_LOGS_DIR_PATH = f"{BASE_DIR}/fit"
GENERATION_LOG = f"{BASE_DIR}/Generations"
TRAIN_SUMMARY_WRITER = tf.summary.create_file_writer(TRAIN_LOGS_DIR_PATH)


def plot_to_buff(generations:np.ndarray, nvertical:int=3, nhoriz:int=3):
    legend = [f"feat {j}" for j in range(generations.shape[-1])]

    fig = plt.figure(figsize=(18, 10))
    plt.suptitle("Generations After GAN Training.")

    for i in range(nvertical* nhoriz):
        ax = plt.subplot(nvertical, nhoriz, i+ 1)
        ax.set_title(f"sequence {i+1}")

        plt.plot(generations[i])
        ax.grid(True)
        plt.legend(legend)

    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close(fig)
    return buf




def log_losses(epoch, plot_buf):
    image = tf.image.decode_png(plot_buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)


    with TRAIN_SUMMARY_WRITER.as_default():
        tf.summary.scalar("Generator Loss", generator_metric.result(), step=epoch)
        tf.summary.scalar("Discriminator Loss", discriminator_metric.result(), step=epoch)
        tf.summary.scalar("Mode Colapsing ?", similarity_metric.result(), step=epoch)
        tf.summary.scalar("Correlation Metric", correlation_metric.result(), step=epoch)

        tf.summary.image("Training Generations", image, step=epoch)


# Training Functions

In [ ]:
def generate_plots(noise, save_to):
    generated = generator(noise)

    fig =plt.figure(figsize=(18, 5))
    plt.title("Generation of the GAN during Training.")
    for i in range(generated.shape[-1]):
        plt.plot(generated[0, :, i], label=f'feat {i+1}')
    plt.grid()
    plt.legend()

    plt.savefig(save_to)
    plt.close(fig)

In [ ]:
@tf.function
def train_step(images):
    alpha= 1  
    noise = tf.random.normal([BS, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)
      extracted_feat = early_predictor(generated_images, training=False)

      sim_loss = similarity_loss(extracted_feat)
      gen_loss = generator_loss(fake_output)+ alpha* sim_loss
      disc_loss = discriminator_loss(real_output, fake_output)


    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


    # Save metric for display
    generator_metric(gen_loss)
    discriminator_metric(disc_loss)
    similarity_metric(sim_loss)

In [ ]:
def train(dataset, epochs):
  losses = []
  for epoch in range(epochs):
    start = time.time()

    generator_metric.reset_states()
    discriminator_metric.reset_states()

    for s, image_batch in enumerate(dataset):
      train_step(image_batch)
      print(f"\r e {epoch}/{epochs}: step {s}", end="")

    stop = time.time()
    print(f"\r[+] Epoch {epoch}/{epochs}: Gen {generator_metric.result():0.4f}; Disc {discriminator_metric.result():0.4f} sim loss: {similarity_metric.result():0.4f} in {(stop-start):0.4f} seconds; {(stop-start)/s:0.4f} s/step")


    # Make generations on seed
    seed_generation = generator(seed, training=False)
    buff = plot_to_buff(seed_generation)

    batch_signature = signature_on_batch(seed_generation, [0, 1], [2, 3, 4, 5], config.met_params.signature_length)
    signature_difference = signature_metric(real_batch_signature, batch_signature)
    correlation_metric(signature_difference)

    l = [generator_metric.result(), discriminator_metric.result()]
    losses.append(l)
    log_losses(epoch, buff)

  return np.array(losses)


In [ ]:
training_losses = train(dset_simulated, EPOCHS)

In [ ]:
generator.save(f"{BASE_DIR}/generator.h5")

## Plot Losses.

In [ ]:
plt.figure(figsize=(18, 5))
plt.title("Training Losses.")
plt.plot(training_losses[:, 0], ".-", label="Generator Loss")
plt.plot(training_losses[:, 1], ".-", label="Discriminator Loss")
plt.grid()
plt.legend()
plt.show()

# Let's Test it!

In [ ]:
after_training_generations = generator(seed, training=False)

plt.figure(figsize=(18, 5))
plt.title("Generation of the GAN whitout Training.")
for i in range(after_training_generations.shape[-1]):
    plt.plot(after_training_generations[0, :, i], label=f'feat {i+1}')
plt.grid()
plt.legend()
plt.show()

In [ ]:
after_training_decision = discriminator(after_training_generations)
after_training_decision[0]

## Test if Mode Colapsing

In [ ]:
# after_training_generations
def plot_several_generations(generations:np.ndarray, nvertical:int=3, nhoriz:int=3):

    legend = [f"feat {j}" for j in range(generations.shape[-1])]

    plt.figure(figsize=(18, 10))
    plt.suptitle("Generations After GAN Training.")

    for i in range(nvertical* nhoriz):
        ax = plt.subplot(nvertical, nhoriz, i+ 1)
        ax.set_title(f"sequence {i+1}")

        plt.plot(generations[i])
        ax.grid(True)
        plt.legend(legend)

    plt.tight_layout()
    plt.show()

plot_several_generations(after_training_generations)